[View in Colaboratory](https://colab.research.google.com/github/LjBLincoln/Machine_Learning/blob/master/wxplusb.ipynb)

# **Create Train Data and Test Data**

In [0]:
import os

import numpy as np 

TRAIN_DATA = 8000
TEST_DATA = 20000

TRAIN_DATA_FILE = 'train'
TEST_DATA_FILE = 'test'

def create_DATA(file):
    if (os.path.isfile(file)):
        x_data = np.loadtxt(file)
    else:
        X = np.random.rand(TRAIN_DATA).astype(np.float32)
        x_data = np.array([X,np.random.uniform(0,3, TRAIN_DATA) + X]).reshape(TRAIN_DATA,2)
        np.savetxt(file,x_data)
    
    #print(x_data)

create_DATA(TRAIN_DATA_FILE)
create_DATA(TEST_DATA_FILE)

# **Create TFRecord **

In [0]:
# -*- coding: utf-8 -*-
import os 
import tensorflow as tf

TRAIN_DATA_FILE = 'train'
TEST_DATA_FILE = 'test'

def _float_feature(value):
    return tf.train.Feature(float_list = tf.train.FloatList(value = [value]))
  
def create_TFRecord(file):
    classes = np.loadtxt(file)

    writer = tf.python_io.TFRecordWriter(file +'.tfrecords')

    for index, data in enumerate(classes):
        print(index,data[0],data[1]) 
        example = tf.train.Example(features = tf.train.Features(feature = {"y_data": _float_feature(data[1]),
                                                                           "x_data": _float_feature(data[0]),                                                                          
                                                                            }))
        writer.write(example.SerializeToString()) 
    writer.close()
  
create_TFRecord(TRAIN_DATA_FILE)
create_TFRecord(TEST_DATA_FILE)

# **Read TFRecord **

In [0]:
import tensorflow as tf


def read_and_decode(filename, batch_size): # read train.tfrecords
    filename_queue = tf.train.string_input_producer([filename])# create a queue

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)#return file_name and file
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'x_data': tf.FixedLenFeature([], tf.float32),
                                           'y_data' : tf.FixedLenFeature([], tf.float32),
                                       })#return image and label

    X_ = tf.cast(features['x_data'], tf.float32)
    Y_ = tf.cast(features['y_data'], tf.float32) 

    x_batch, y_batch = tf.train.shuffle_batch([X_, Y_],
                                                    batch_size= batch_size,
                                                    num_threads=64,
                                                    capacity=2000,
                                                    min_after_dequeue=1500,
                                                    )
    return x_batch, tf.reshape(y_batch,[batch_size])


# **Show TFRecords**

In [0]:
import numpy as np 

tfrecords_file = 'train.tfrecords'

BATCH_SIZE = 8
x_batch, y_batch = read_and_decode(tfrecords_file, BATCH_SIZE)

with tf.Session()  as sess:

    i = 0
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    try:
        while not coord.should_stop() and i<8000:
            # just plot one batch size
            x, y = sess.run([x_batch, y_batch])
            for j in np.arange(BATCH_SIZE):
                print(x)
                print(y)
            i+=1
    except tf.errors.OutOfRangeError:
        print('done!')
    finally:
        coord.request_stop()
    coord.join(threads)


In [0]:
classes = np.loadtxt('train.txt')

for index, x_data in enumerate(classes):
    print(index,x_data[0],x_data[1])
    

# Demo (y = wx+b)

In [0]:
# _*_ coding: utf-8 _*_

import matplotlib.pyplot as plt

import tensorflow as tf

import numpy as np

import os

MAX_STEPS = 1000

NUM_EXAMPLES = 10000

MIN_X = 0
MIN_Y = 0
MAX_X = 1
MAX_Y = 1


if (os.path.isfile('train.txt')):
    x_data = np.loadtxt('train.txt')
else:
    X = np.random.rand(NUM_EXAMPLES).astype(np.float32).reshape(1,100)
    x_data = np.array([X,np.random.uniform(MIN_Y,MAX_Y, NUM_EXAMPLES)] + X )
    np.savetxt('train.txt',x_data)
    
if (os.path.isfile('test.txt')):
    x_data = np.loadtxt('test.txt')
else:
    X = np.random.rand(NUM_EXAMPLES).astype(np.float32)
    x_data = np.array([X,np.random.uniform(MIN_Y,MAX_Y, NUM_EXAMPLES)] + X )
    np.savetxt('test.txt',x_data)

#input
with tf.name_scope('input'):
    with tf.name_scope('x'):
        x = tf.placeholder(tf.float32, shape = (100,), name = "x")
    with tf.name_scope('y_'):
        y_ = tf.placeholder(tf.float32, shape = (100,), name = "y_")

# layer
with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        ###对权进行赋值 在-1到1之间随机数
        Weights = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
        tf.summary.histogram('Weights' ,Weights)
    with tf.name_scope('biases'):
        #初始偏差为零
        biases = tf.Variable(tf.zeros([1]))
        tf.summary.histogram('biases' ,biases)
    with tf.name_scope('Wx_plus_b'):
        #权值与x相乘并加偏差
        y = Weights * x + biases 

#Mean Squared Error)
with tf.name_scope('Mean_Squared_Error'):
    #方差，(y-y_)平方，求和，取均值
    loss = tf.reduce_mean(tf.square(y-y_))
    tf.summary.scalar('loss', loss)

#Optimizer
with tf.name_scope('train'):
    #定义梯度下降法优化函数优化，步长为0.5
    optimizer = tf.train.GradientDescentOptimizer(0.1)
    #tf.summary.scalar('optimizer', optimizer)
    train_step = optimizer.minimize(loss)

  
def feed_dict():
    xs, ys =x_data[0], x_data[1]
    return {x:xs, y_:ys}


merged = tf.summary.merge_all()

init = tf.global_variables_initializer()

#session
sess = tf.Session()

#create FileWriter and loadd graph
train_writer = tf.summary.FileWriter('./log', sess.graph)

sess.run(init)

plt.figure(figsize=(10, 10), dpi=80)
plt.ion()

#Save
saver = tf.train.Saver()

for step in range(MAX_STEPS):
    
    plt.title(" Diagram.")
    plt.xlabel("x_data")
    plt.ylabel("y_")
    plt.xlim(xmax=MAX_X, xmin=MIN_X)
    plt.ylim(ymax=MAX_Y,ymin=MIN_Y)
    plt.plot(x_data[0],x_data[1],'ro',color='b',linewidth=0.5)

    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    summary,y_re,loss_value, _ = sess.run([merged,y,loss,train_step],feed_dict=feed_dict(),options=run_options)
    train_writer.add_summary(summary, step)
    print(feed_dict())
    if step % 50 == 0:
      print("After %d training step(s), loss on training batch is %g." %(step, loss_value))
      saver.save(sess,'./log/models/mnist_model.ckpt', global_step=step)
    
    
    if step % 50 == 0:
        plt.plot(x_data[0],y_re,'ro', color='g',linewidth=0.5)
    if loss_value <= 0.0863299:
        plt.plot(x_data[0],y_re,'ro', color='r',linewidth=0.5)
        


plt.ioff()
plt.show()
    
#close FileWriter
train_writer.close()
sess.close()

print("Done !!!")

#https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

LOG_DIR = './log'
get_ipython().system_raw('tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(LOG_DIR))

get_ipython().system_raw('./ngrok http 6006 &')


! curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"



In [0]:
import matplotlib.pyplot as plt

import numpy as np

MAX_STEPS = 500

NUM_EXAMPLES = 10000

# create data 生成100个0-1之间的随机数   np.random.rand(100) 1*100的矩阵，其每个元素为0-1的随机数
#x_data = np.random.rand(NUM_EXAMPLES).astype(np.float32)

#y_data = x_data

#x_data =np.array([np.random.uniform(1,1.2, NUM_EXAMPLES),np.random.uniform(1,1.2, NUM_EXAMPLES)]) #np.array([np.linspace(1, 4, NUM_EXAMPLES), np.linspace(3, 9, NUM_EXAMPLES)])
X = np.random.rand(NUM_EXAMPLES).astype(np.float32)
#print(X)
x_data = np.array([X,np.random.uniform(0,1, NUM_EXAMPLES)] + X ).reshape(NUM_EXAMPLES,2)
print(x_data)

plt.figure(figsize=(10, 10), dpi=80)
plt.ion()
plt.title(" Diagram.")
plt.xlabel("x_data")
plt.ylabel("y_")
#plt.xlim(xmax=1, xmin=0)
#plt.ylim(ymax=1,ymin=0)
plt.plot(x_data[0:1],x_data[0:1],'ro',color='r',linewidth=0.5)
plt.ioff()
plt.show()

In [0]:
!ls -la